## Hands-on
### Quantifying and Reducing Gender Stereotypes in Portuguese Word Embeddings

Based in the following paper: Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings by Tolga Bolukbasi, Kai-Wei Chang, James Zou, Venkatesh Saligrama, and Adam Kalai. Proceedings of NIPS 2016.

In [33]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions_pt

### Part1 
#### Gender Bias in Word Embedding


- #### Step 1: Load data

In [34]:
# load google news word2vec
E = WordEmbedding('./embeddings/cbow3_s300.txt')
# load professions
professions = load_professions_pt()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/cbow3_s300.txt
(24982, 300)
24982 words of dimension 300 : ., a, o, e, ..., decolar, discursou, sinai, ldu
24982 words of dimension 300 : ., a, o, e, ..., decolar, discursou, sinai, ldu
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


- #### Step 2: Define Gender Direction

In [41]:
# gender direction
v_gender = E.diff('mulher', 'homem')

- #### Step 3:Generating analogies of "Homem: x :: Mulher : y"

In [42]:
a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 11.634296693619406
Median: 5.0
feminino-masculino
acusadas-acusados
quisesse-queria
agremiações-atrações
viu-achou
alves-pires
templo-prédio
ouvidas-julgados
declaram-alegaram
agarrar-puxar
quiser-quer
veem-acharam
detidas-indiciados
identificadas-identificados
identificar-localizar
mortas-assassinados
queira-quis
resgatado-espancado
recusam-recusou-se
pensam-argumentam
fabricantes-distribuidores
associar-obrigar
pensamentos-méritos
libertou-expulsou
judaicas-europeias
previstas-estabelecidas
avaí-américa-mg
governa-governou
santidade-excelência
importam-importava
esconder-aguentar
sons-arranjos
detalhou-informou
envolvam-conduzem
disser-garanto
coordena-informa
propõem-permitem
acreditam-supõe-se
vêem-apareciam
dedicação-delicadeza
estiver-fica
esquecido-tirado
concepções-realizações
fórmulas-características
presas-presos
haxixe-heroína
libertada-atacada
influentes-destacados
túmulo-apartamento
casarão-quarteirão
entusiasta-admirador
peregrinos-marinheiros
pu

- #### Step 4: Analyzing gender bias in word vectors asscoiated with professions

In [43]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.056924507, 'administradora'),
  (-0.03283116, 'arquiteto'),
  (-0.018745277, 'assassina'),
  (-0.012723602, 'atriz'),
  (-0.001701042, 'contador'),
  (0.0062553417, 'advogada'),
  (0.011022608, 'analista'),
  (0.016029747, 'ajudante'),
  (0.019969206, 'aventureiro'),
  (0.028090533, 'arcebispo'),
  (0.029963575, 'ator'),
  (0.0391017, 'conhecida'),
  (0.042194746, 'vereadora'),
  (0.045008607, 'embaixador'),
  (0.06080038, 'antropólogo'),
  (0.062282346, 'arqueólogo'),
  (0.069254465, 'artista')],
 [(-0.056924507, 'administradora'),
  (-0.03283116, 'arquiteto'),
  (-0.018745277, 'assassina'),
  (-0.012723602, 'atriz'),
  (-0.001701042, 'contador'),
  (0.0062553417, 'advogada'),
  (0.011022608, 'analista'),
  (0.016029747, 'ajudante'),
  (0.019969206, 'aventureiro'),
  (0.028090533, 'arcebispo'),
  (0.029963575, 'ator'),
  (0.0391017, 'conhecida'),
  (0.042194746, 'vereadora'),
  (0.045008607, 'embaixador'),
  (0.06080038, 'antropólogo'),
  (0.062282346, 'arqueólogo'),
  (0.069254

### Part 2
#### Debias

- #### Step 5:

In [38]:
from debiaswe.debias import debias

In [39]:
with open('./data/definitional_pairs_pt.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs_pt.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed_pt.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['mulher', 'homem'], ['garota', 'garoto'], ['ela', 'ele'], ['mãe', 'pai'], ['filha', 'filho'], ['moça', 'moço'], ['fêmea', 'macho'], ['dela', 'dele'], ['própria', 'próprio']]
gender specific 158 ['atriz', 'atrizes', 'tia', 'tias', 'bacharel', 'bacharela', 'bailarina', 'barbearia', 'barítono', 'barba']


In [40]:
debias(E, gender_specific_words, defs, equalize_pairs)

sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

24982 words of dimension 300 : ., a, o, e, ..., decolar, discursou, sinai, ldu
{('MACHO', 'FÊMEA'), ('PAIS', 'MÃES'), ('filhos', 'filhas'), ('MONASTÉRIO', 'CONVENTO'), ('PRÓPRIO', 'PRÓPRIA'), ('Irmão Gêmeo', 'Irmã Gêmea'), ('Tio', 'Tia'), ('Pais', 'Mães'), ('Netos', 'Netas'), ('IRMÃO GÊMEO', 'IRMÃ GÊMEA'), ('monastério', 'convento'), ('HOMEM', 'MULHER'), ('Vereador', 'Vereadora'), ('Dele', 'Dela'), ('ele', 'ela'), ('rei', 'rainha'), ('presidente', 'presidenta'), ('Câncer De Próstata', 'Câncer De Ovário'), ('pai', 'mãe'), ('SOBRINHO', 'SOBRINHA'), ('aluno', 'aluna'), ('MENINOS', 'MENINAS'), ('Vô', 'Vó'), ('Machos', 'Fêmeas'), ('TIO', 'TIA'), ('PAI', 'MÃE'), ('NETOS', 'NETAS'), ('REI', 'RAINHA'), ('TESTOSTERONA', 'ESTROGÊNIO'), ('Ele', 'Ela'), ('paternidade', 'maternidade'), ('Empresário', 'Empresária'), ('cavalheiro', 'dama'), ('reis', 'rainhas'), ('pais', 'mães'), ('MENINO', 'MENINA'), ('Avô', 'Avó'), ('câncer de próstata', 'câncer de ovário'), ('VEREADOR', 'VEREADORA'), ('cavalheiros'

([(-0.06245242, 'aventureiro'),
  (-0.03370887, 'artista'),
  (-0.033379897, 'assassina'),
  (-0.019246414, 'ator'),
  (-0.018068824, 'arqueólogo'),
  (-0.009368014, 'analista'),
  (-0.0075146593, 'advogada'),
  (0.014118314, 'conhecida'),
  (0.020851301, 'contador'),
  (0.021441298, 'administradora'),
  (0.03413377, 'embaixador'),
  (0.0379627, 'antropólogo'),
  (0.045062304, 'arquiteto'),
  (0.04605992, 'arcebispo'),
  (0.07386991, 'ajudante'),
  (0.3055688, 'atriz'),
  (0.40745574, 'vereadora')],
 [(-0.06245242, 'aventureiro'),
  (-0.03370887, 'artista'),
  (-0.033379897, 'assassina'),
  (-0.019246414, 'ator'),
  (-0.018068824, 'arqueólogo'),
  (-0.009368014, 'analista'),
  (-0.0075146593, 'advogada'),
  (0.014118314, 'conhecida'),
  (0.020851301, 'contador'),
  (0.021441298, 'administradora'),
  (0.03413377, 'embaixador'),
  (0.0379627, 'antropólogo'),
  (0.045062304, 'arquiteto'),
  (0.04605992, 'arcebispo'),
  (0.07386991, 'ajudante'),
  (0.3055688, 'atriz'),
  (0.40745574, 'vere